# Presentation of the code

Import package fipy: ('\*'means import all functions).
Import random for the initial condition of the interface.

In [1]:
from fipy import *
import random

## Geometry and mesh

We first install the geometry of the situation.
The simulation is 2D (W=1., L=1.).

### Space

In [4]:
L = 1. #length
W = 1. #width: characteristic length
b = 1. #gap

### Mesh

The mesh is decided with the length of one controle volume. (besoin de étude de convergence)

In [5]:
#Mesh
dx = 0.25 #width of controle volume
dy = 0.10
nx = 100
ny = 250 #number of controle volume
mesh = Grid2D(dx=dx, dy=dy, nx=nx, ny=ny)

## Description of the fluids

### Parameters

Fluid 1 is the active fluid, less viscous. phi = 0. viscosity1=0.1.

Fluid 2 is the passive fluid, more viscous. phi = 1. viscosity2=1. 

The two permeabilities are set to 1.

We introduce the parameter beta, easier to use.

In [6]:
viscosity2 = 1.
Mobility = 0.1 #ratio of the two viscosities
viscosity1 = viscosity2 * Mobility
permeability1 = permeability2 = 1.
beta1 = viscosity1 / permeability1
beta2 = viscosity2 / permeability2

### Variable of the fluids

The grid is colocated in FiPy. We need to introduce a correction in the velocity to suppress oscillations. Therefore we need two variables of speed.

We have here two coupled equations: in order to the variables to respect both at each time step, we need an iterative scheme with a correction of the pressure at each iteration. For this correction, we use the SIMPLE algorithm.
We introduce a new pressure variable that corresponds to the pressure once it has been corrected by the second equation, that will be used to begin again the iteration.

In [7]:
pressure = CellVariable(mesh=mesh, name='pressure')
pressureCorrection = CellVariable(mesh=mesh)
xVelocity = CellVariable(mesh=mesh, name='X Velocity')
yVelocity = CellVariable(mesh=mesh, name='Y Velocity')
velocity = FaceVariable(mesh=mesh, rank=1)

## Phase-field model

### Order parameter

The order parameter is phi. The hasOld option is important, because the phase-field equation is not linear so we need to iterate the solving of the equation at each time step. The commad hasOld permits to update the value of phi to use at the end of the iterations only.

In [8]:
phi = CellVariable(name=r'$\phi$', mesh=mesh, hasOld=1)

### New values

The phase-field method means that the two fluids are not separated anymore: this means all the variable have to be defined on the whole domain. The particular variables are transformed by interpolation.

In [9]:
beta = CellVariable(mesh=mesh, name='beta', value = beta2 * phi + beta1 * (1-phi), hasOld=1)

### Parameters of Phase field equation

The Cahn number is defined thanks to the values of the paper for the moment. Justification?
l is chosen as 1 because its impact is still to define.

In [10]:
#Cahn_number = 0.001
epsilon = 0.1
M = Mobility * epsilon**2
l = 1.

### Phase-field equation

The equation chosen for the phase-field is base on Cahn-Hilliard model as described in the paper. (need of a better explanation here)
The first line is the creation of an interpolation of the value of phi on to the faces. FiPy would have it done automatically, but it is more accurate to do it by arithmeticFaceValue (We also get to choose what type of interpolation we use) (why arithmetic?? I think because more precise is not needed, but I need to review, not the priority though)

In [11]:
PHI = phi.arithmeticFaceValue #result more accurate by non-linear interpolation
coeff1 = Mobility * l * (6.* PHI*(PHI-1.) + 1.)
eq = (TransientTerm() + ConvectionTerm(velocity) == DiffusionTerm(coeff=coeff1) - DiffusionTerm(coeff=(M, l)))

## Pressure and velocity

### Equations

We then introduce the equations: equation of motion, here Darcy's law and equation of continuity, permitting the correction of the pressure. (need to precise this point to explain better)

In [12]:
xVelocityEq = (ImplicitSourceTerm(coeff=beta) + pressure.grad.dot([1.,0.]))
yVelocityEq = (ImplicitSourceTerm(coeff=beta) + pressure.grad.dot([0.,1.]))

In [13]:
ap = CellVariable(mesh=mesh, value=1.)
coeff = 1./ ap.arithmeticFaceValue * mesh._faceAreas * mesh._cellDistances
X = mesh.cellCenters[0]
pressureCorrectionEq = DiffusionTerm(coeff=coeff) - velocity.divergence

### Rhie-Chow correction

FiPy uses a colocated grid, causing oscillations in the velocity field. It is necessary to apply a correction.

In [14]:
#Remove oscillations
from fipy.variables.faceGradVariable import _FaceGradVariable
volume = CellVariable(mesh=mesh, value=mesh.cellVolumes, name='Volume')
contrvolume=volume.arithmeticFaceValue

## Boundary conditions

### Phase

The first line is the outflow condition on the right.

Phi is set up as a crenau function in the beginning. We set up a random initial condition in the position of the front phase so that we have a small random perturbation that permits the formation of fingers.
'x,y = mesh.cellCenters' is to define x as the first axis so we can define the boundary conditions.

In [15]:
phi.faceGrad.constrain([0.], mesh.facesRight)

x,y = mesh.cellCenters
def initialize(phi):
    phi.setValue(0.)
    for i in range(ny):
        a = random.gauss(0.1, 0.01)
        phi.setValue(1., where=(x > nx*dx * a ) & (y<(i+1)*dy) & (y>(i*dy)))

initialize(phi)

### Velocity and pressure

We decide the rate of injection of the first fluid.
The boundary conditions are set on the velocity so that the rate is respected and the quantity of volume is conserved (incompressible fluids).

In [16]:
Q = 1. #rate of injection
#U = Q / (b*W)
U = 0.8 #if more, it gets unstable, I should change the time step

## Initialization

We need to initialize the phase-field to separate the effcet of forming from the effect of moving it. For this, we need to have the initial fields of the pressure and the velocity.

We introduce functions so we will be able to speed the code by compiling it.

### Phase field

The time step here is only used to resolve the equation. But the iteration here are not time steps, but iteartion to getr a more accurate solution: by using the new value obtained at the solving of the equation, we are closer to the solution, so by solving it again with this value, we get a more little residual, a more accurate solution. (we have a descritez solution, so the closer the values are when solving, the solution gets more accurate : by convergence.) (need to get this clearer)

In [17]:
#Phase
timeStep = 10.

    
def updatephi():
    phi.updateOld()
    beta.updateOld()
    res = 1e+10
    while res > 1e-10:
        res = eq.sweep(var=phi, dt=timeStep)


for i in range(50):
    updatephi()

KeyboardInterrupt: 

### Pressure and velocity fields

We use the SIMPLE algorithm that we will use at each iteration.
The relaxation are kept the same as in the example. (need to review the exact utility)

In [ ]:
#Pressure and velocity
pressureRelaxation = 0.8
velocityRelaxation = 0.5
X, Y = mesh.faceCenters
xVelocity.constrain(U, mesh.facesLeft)
#xVelocity.constrain(U, mesh.facesRight)
pressureCorrection.constrain(0., mesh.facesRight)
sweeps = 10

    
def velopres():
    ##Solve the Stokes equations to get starred values
    xVelocityEq.cacheMatrix()
    xres = xVelocityEq.sweep(var=xVelocity, underRelaxation=velocityRelaxation)
    xmat = xVelocityEq.matrix
    yres = yVelocityEq.sweep(var=yVelocity, underRelaxation=velocityRelaxation)
    ##update the ap coefficient from the matrix diagonal
    ap[:] = xmat.takeDiagonal()
    #
    ##update the face velocities based on starred values with the Rhi-Chow correction
    #cell pressure gradient
    presgrad = pressure.grad
    #face pressure gradient
    facepresgrad = _FaceGradVariable(pressure)
    #
    velocity[0] = xVelocity.arithmeticFaceValue + contrvolume / ap.arithmeticFaceValue * (presgrad[0].arithmeticFaceValue-facepresgrad[0])
    velocity[1] = yVelocity.arithmeticFaceValue + contrvolume / ap.arithmeticFaceValue * (presgrad[1].arithmeticFaceValue-facepresgrad[1])
    #velocity[..., mesh.exteriorFaces.value]=0.
    #velocity[0].constrain(U, mesh.facesRight | mesh.facesLeft)
    #velocity[0, mesh.facesLeft.value] = U
    #velocity[0, mesh.facesRight.value] = U
    #
    ##solve the pressure correction equation
    pressureCorrectionEq.cacheRHSvector()
    ## left bottom point must remain at pressure 0, so no correction
    pres = pressureCorrectionEq.sweep(var=pressureCorrection)
    rhs = pressureCorrectionEq.RHSvector
    #
    ## update the pressure using the corrected value
    pressure.setValue(pressure + pressureRelaxation * pressureCorrection)
    ## update the velocity using the corrected pressure
    xVelocity.setValue(xVelocity - pressureCorrection.grad[0] / ap * mesh.cellVolumes)
    yVelocity.setValue(yVelocity - pressureCorrection.grad[1] / ap * mesh.cellVolumes)
    xVelocity[0]=U
#    xVelocity[nx-1]=U
#    if sweep%10 == 0:
#        viewer2.plot()

for sweep in range(sweeps):
    velopres()

## Dynamics

In [ ]:
x = mesh.cellCenters[0]    

displacement = 12.
#velocity1 = 1.
timeStep = .1 * dx / U
elapsed = 0.
while elapsed < displacement/U:
    updatephi()
    elapsed +=timeStep
    for sweep in range(sweeps):
        velopres()

## Viewers

In [ ]:
viewer = Viewer(vars = (phi,), datamin=0., datamax=1.)
viewer.plot()    
viewer2 = Viewer(vars = (xVelocity, yVelocity), datamin=-1., datamax=3.)
viewer2.plot()

This last line stops the code at the end so we are able to see the results.

In [ ]:
raw_input("pause")